## IMPORTAR LAS LIBRERIAS

Actualizar a las que se usen finalmente en tu proyecto.

In [35]:
import numpy as np
import pandas as pd
import cloudpickle

#Automcompletar rápido
%config IPCompleter.greedy=True

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

## CARGAR LOS DATOS

### Ruta del proyecto

In [2]:
ruta_proyecto = 'C:/Users/110927/Desktop/EstructuraDirectorio/03_MACHINE_LEARNING/CASOS/00_LeadScoring'

### Nombre del fichero de datos

In [3]:
nombre_fichero_datos = 'Leads.csv'

### Cargar los datos

In [10]:
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

df = pd.read_csv(ruta_completa,index_col='id', sep=';')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9093 entries, 660737 to 579533
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   origen                 9093 non-null   object 
 1   fuente                 9058 non-null   object 
 2   no_enviar_email        9093 non-null   object 
 3   no_llamar              9093 non-null   object 
 4   compra                 9093 non-null   int64  
 5   visitas_total          8970 non-null   float64
 6   tiempo_en_site_total   9093 non-null   int64  
 7   paginas_vistas_visita  8970 non-null   float64
 8   ult_actividad          9003 non-null   object 
 9   ambito                 7655 non-null   object 
 10  ocupacion              6405 non-null   object 
 11  conociste_google       9093 non-null   object 
 12  conociste_revista      9093 non-null   object 
 13  conociste_periodico    9093 non-null   object 
 14  conociste_youtube      9093 non-null   object 
 15  co

### Seleccionar solo las variables finales

#### Cargar la lista de variables finales

In [12]:
nombre_variables_finales = ruta_proyecto + '/05_Resultados/' + 'variables_finales.pickle'

pd.read_pickle(nombre_variables_finales).sort_index().index.to_list()

['ambito_Marketing Management',
 'ambito_OTROS',
 'ambito_Select',
 'ocupacion_Unemployed',
 'ocupacion_Working Professional',
 'paginas_vistas_visita_mms',
 'score_actividad_mms',
 'score_perfil_mms',
 'tiempo_en_site_total_mms',
 'ult_actividad_Chat Conversation',
 'ult_actividad_Converted to Lead',
 'ult_actividad_Page Visited on Website',
 'ult_actividad_SMS Sent',
 'visitas_total_mms']

#### Apuntar (manualmente) la lista de variables finales sin extensiones

In [26]:
variables_finales = ['ambito',
                     'ocupacion',
                     'ult_actividad',
                     'paginas_vistas_visita',
                     'score_actividad',
                     'score_perfil',
                     'tiempo_en_site_total',
                     'visitas_total']

#### Crear la matriz de variables procesos (excel)

Ir a la plantilla de Excel "Fase Producción Plantilla Procesos" y crear la matriz de variables por procesos.

#### Actualizar las importaciones

Ir arriba a la celda de importacion de paquetes y actualizarlos con los que finalmente vamos a usar.

## ESTRUCTURA DE LOS DATASETS

### Eliminar registros

#### Por duplicados

In [12]:
df.drop_duplicates(inplace = True)

In [13]:
df=df.loc[(df.no_enviar_email!='Yes')&(df.ult_actividad!='Email Bounced')&(df.no_llamar!='Yes')]

#### Para x

Quedarse solo con las de la lista.

In [27]:
x = df[variables_finales].copy()

#### Para y

Especificar la target.

In [44]:
target = 'compra'

Crear el y.

In [45]:
y = df[target].copy()

## CREAR EL PIPELINE

### Instanciar calidad de datos

#### Crear la función

In [48]:
x

,ambito,ocupacion,ult_actividad,paginas_vistas_visita,score_actividad,score_perfil,tiempo_en_site_total,visitas_total
id,,,,,,,,
660737,Select,Unemployed,Page Visited on Website,0.0,15.0,15.0,0,0.0
660728,Select,Unemployed,Email Opened,2.5,15.0,15.0,674,5.0
660727,Business Administration,Student,Email Opened,2.0,14.0,20.0,1532,2.0
660719,Media and Advertising,Unemployed,Unreachable,1.0,13.0,17.0,305,1.0
660681,Select,Unemployed,Converted to Lead,1.0,15.0,18.0,1428,2.0
...,...,...,...,...,...,...,...,...
579622,Select,Unemployed,SMS Sent,2.6,NaN,NaN,1409,13.0
579615,Business Administration,Unemployed,SMS Sent,2.5,14.0,20.0,210,5.0
579546,Media and Advertising,Unemployed,SMS Sent,2.0,14.0,19.0,238,2.0


In [49]:
def calidad_datos(df):
    temp = df.astype({'visitas_total': 'Int64'})             
    #imputar por valor
    valor='DESCONOCIDO'
    var_imputar_valor = ['ult_actividad']
    temp[var_imputar_valor]=temp[var_imputar_valor].fillna(valor)
    #atipicos numericas, winsorizacion manual
    var_winsor=['visitas_total']    
    temp[var_winsor].clip(0,50, inplace=True)
    #imputar por moda
    def imputar_moda(variable):
        return(variable.fillna(variable.mode()[0]))
    var_imputar_moda = ['ocupacion','ambito']
    temp[var_imputar_moda] = temp[var_imputar_moda].apply(imputar_moda)
    #imputar por mediana
    def imputar_mediana(variable):
        if pd.api.types.is_integer_dtype(variable):
            return(variable.fillna(int(variable.median())))
        else:
            return(variable.fillna(variable.median()))
    var_imputar_mediana = ['score_actividad','score_perfil','visitas_total','paginas_vistas_visita']
    temp[var_imputar_mediana] = temp[var_imputar_mediana].apply(imputar_mediana)
    #imputar cat raras
    def agrupar_cat_raras(variable, criterio = 0.05):
        #Calcula las frecuencias
        frecuencias = variable.value_counts(normalize=True)
        #Identifica las que están por debajo del criterio
        temp = [cada for cada in frecuencias.loc[frecuencias < criterio].index.values]
        #Las recodifica en 'OTROS'
        temp2 = np.where(variable.isin(temp),'OTROS',variable)
        #Devuelve el resultado
        return(temp2)
    var_imputar_cat_raras = ['ambito','ocupacion','ult_actividad']
    for variable in var_imputar_cat_raras:
        temp[variable] = agrupar_cat_raras(temp[variable],criterio = 0.02)
    
    
    return(temp)

#### Convertirla en transformer

In [50]:
hacer_calidad_datos = FunctionTransformer(calidad_datos)

### Instanciar transformación de variables

In [51]:
var_ohe = ['ambito', 'ocupacion', 'ult_actividad']
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')
var_mms = ['paginas_vistas_visita','score_actividad',
         'score_perfil','tiempo_en_site_total','visitas_total']
mms = MinMaxScaler()

### Crear el pipe del preprocesamiento

#### Crear el column transformer

In [52]:
ct = make_column_transformer(
    (ohe, var_ohe),
    ( mms,var_mms),
    remainder='drop')

#### Crear el pipeline del preprocesamiento

In [53]:
pipe_prepro = make_pipeline(hacer_calidad_datos, 
                            ct)

### Instanciar el modelo

#### Instanciar el algoritmo

In [54]:
modelo = LogisticRegression(n_jobs = -1, solver='saga', penalty= 'none', C=0.25)
                       

#### Crear el pipe final de entrenamiento

In [55]:
pipe_entrenamiento = make_pipeline(pipe_prepro,modelo)

#### Guardar el pipe final de entrenamiento

In [56]:
nombre_pipe_entrenamiento = 'pipe_entrenamiento.pickle'

ruta_pipe_entrenamiento = ruta_proyecto + '/04_Modelos/' + nombre_pipe_entrenamiento

with open(ruta_pipe_entrenamiento, mode='wb') as file:
   cloudpickle.dump(pipe_entrenamiento, file)

#### Entrenar el pipe final de ejecución

In [57]:
pipe_ejecucion = pipe_entrenamiento.fit(x,y)

C:\Users\110927\AppData\Local\Temp\ipykernel_19392\3507182111.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[var_winsor].clip(-20,10, inplace=True)
C:\Users\110927\AppData\Local\Temp\ipykernel_19392\3507182111.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[var_winsor].clip(-20,10, inplace=True)
C:\Users\110927\AppData\Local\miniconda3\envs\leadscoring\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\U

## GUARDAR EL PIPE

### Nombre del pipe final de ejecución

In [58]:
nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'

### Guardar el pipe final de ejecución

In [59]:
ruta_pipe_ejecucion = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion

with open(ruta_pipe_ejecucion, mode='wb') as file:
   cloudpickle.dump(pipe_ejecucion, file)